In [1]:
import pandas as pd
import numpy as np
import folium
import pickle
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPool2D
from tensorflow import expand_dims
from tensorflow.keras import optimizers, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from sklearn.metrics import r2_score

The following worksheet shows some of my work in trying to develop a convolutional neural network to identify earthquake locations from accelerometer readings. As a full disclaimer, I tried many more network shapes and settings than are present below -- I just tended to twiddle while assembling and and compiling the model due to the difficulties of gridsearching over an NN. Since nothing really outperformed the mean, I wasn't really saving a lot of my tests for reference.

In [2]:
final_df=pd.read_pickle("..\..\data\pickled_files\\top_1000_final.p")

In [3]:
devices=pd.read_csv("..\..\data\devices.csv")

I also got rid of the outliers, as they cause issues in even the most basic predictions.

In [4]:
final_df=final_df[final_df['outlier']==0]

In [5]:
final_df

,magnitude,latitude,longitude,depth,Referencia de localizacion,utc_datetime,outlier,device_1,device_2,device_4,...,device_22,device_23,device_24,device_26,device_27,device_29,device_33,device_999,state,state_lab
0,7.4,15.8033,-96.1337,22.8,"4 km al NORTE de CRUCECITA, OAX",2020-06-23 15:29:03,0,"[[0.0021874999999999998, -0.014062499999999999...","[[-0.007187500000000001, -0.018125, -0.024375,...","[[0.0034375, -0.004374999999999999, 0.01000000...",...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.007187500000000001, -0.005625000000000001,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",OAX,10
1,7.2,16.2180,-98.0135,16.0,"14 km al SURESTE de PINOTEPA NACIONAL, OAX",2018-02-16 23:39:39,0,"[[0.09006250000000002, 0.08934375, 0.10725, 0....","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0016562500000000006, 0.01203125, 0.0146562...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",OAX,10
2,6.5,14.5818,-92.4807,76.2,"37 km al SUROESTE de CD HIDALGO, CHIS",2019-02-01 16:14:12,0,"[[-0.0409375, -0.0340625, -0.0234375, -0.03, -...","[[0.0134375, 0.016875, 0.015312500000000001, 0...","[[0.1409375, 0.1465625, 0.15031250000000002, 0...",...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0871875, 0.09312500000000001, 0.0690625, 0...",CHIS,3
3,6.4,13.7792,-93.3382,9.4,"162 km al SUROESTE de CD HIDALGO, CHIS",2019-11-20 04:27:06,0,"[[-0.0009375000000000004, -0.00343749999999999...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.004375, 0.00031250000000000006, 0.0203125,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-0.0009375000000000008, 0.0075, 0.0031249999...","[[-0.010624999999999999, -0.00375, -0.0190625,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",CHIS,3
6,6.0,16.2527,-94.6173,80.8,"33 km al SURESTE de UNION HIDALGO, OAX",2020-01-05 04:40:49,0,"[[-0.002499999999999999, -0.004375000000000001...","[[0.00375, -0.004375, 0.0009374999999999982, -...","[[0.006562500000000001, 0.0021875, -0.00656250...",...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.014374999999999999, 0.00375, -0.003125, 0....","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0012500000000000002, 0.0084375, 0.00499999...","[[-0.003125000000000001, 0.0309375, -0.0093750...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",OAX,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,4.3,15.5190,-94.9390,21.5,"78 km al SURESTE de SALINA CRUZ, OAX",2019-03-28 09:08:36,0,"[[-0.0359375, -0.04875, -0.044062500000000004,...","[[-0.053750000000000006, -0.0640625, -0.06125,...","[[0.11499999999999999, 0.0990625, 0.103125, 0....",...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

I also added a categorical variable identifying "major" (>=5.5 magnitude) earthquakes for testing for those. Results were underwhelming and aren't included.

In [6]:
final_df['major']=final_df['magnitude'].map(lambda x: 1 if x >= 5.5 else 0)

In [7]:
device_cols=final_df.columns[final_df.columns.str.startswith('device')]

In [8]:
X= final_df[device_cols]

In [9]:
X

,device_1,device_2,device_4,device_5,device_6,device_7,device_8,device_9,device_10,device_11,...,device_20,device_21,device_22,device_23,device_24,device_26,device_27,device_29,device_33,device_999
0,"[[0.0021874999999999998, -0.014062499999999999...","[[-0.007187500000000001, -0.018125, -0.024375,...","[[0.0034375, -0.004374999999999999, 0.01000000...","[[0.0046875, 0.0012499999999999985, -0.0068749...","[[-1.734723475976807e-18, 0.0075, -0.0075, 0.0...","[[0.0121875, 0.009687500000000002, -0.0184375,...","[[-0.029062499999999998, -0.013125000000000001...","[[-0.006874999999999999, 0.0024999999999999996...","[[0.0125, -0.0006249999999999988, -0.002499999...","[[-0.0084375, -0.0009374999999999974, -0.00781...",...,"[[0.0040625, -0.0021874999999999993, -0.003125...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.007187500000000001, -0.005625000000000001,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,"[[0.09006250000000002, 0.08934375, 0.10725, 0....","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.08034375, 0.09371874999999999, 0.102625, 0...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.02925, 0.054218749999999996, 0.04671875, 0...","[[0.16609375, 0.19559375, 0.17381249999999998,...","[[0.1868125, 0.184, 0.19134375, 0.18590625, 0....","[[0.22359375, 0.2315625, 0.2345625, 0.23721875...",...,"[[0.25896875, 0.25928125, 0.2538125, 0.2502187...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0016562500000000006, 0.01203125, 0.0146562...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,"[[-0.0409375, -0.0340625, -0.0234375, -0.03, -...","[[0.0134375, 0.016875, 0.015312500000000001, 0...","[[0.1409375, 0.1465625, 0.15031250000000002, 0...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.021875000000000002, -0.015312500000000001,...","[[-0.012812500000000001, -0.014374999999999999...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0325, 0.002187500000000001, -0.00062500000...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.014687500000000001, 0.017187499999999998, ...",...,"[[-0.012187499999999999, -0.0196875, -0.024375...","[[0.27468750000000003, 0.2634375, 0.2684375, 0...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0871875, 0.09312500000000001, 0.0690625, 0..."
3,"[[-0.0009375000000000004, -0.00343749999999999...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-0.003125000000000001, -0.013125000000000001...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.020625000000000004, -0.015625, -0.00031249...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-0.014687500000000003, -0.019062500000000003...",...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.004062500000000001, 0.004375000000000001, ...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

In [10]:
y=final_df[['latitude', 'longitude']]

In [11]:
y

,latitude,longitude
0,15.8033,-96.1337
1,16.2180,-98.0135
2,14.5818,-92.4807
3,13.7792,-93.3382
6,16.2527,-94.6173
...,...,...
995,15.5190,-94.9390
996,14.5795,-93.7258
997,14.4045,-93.9323
998,19.0497,-107.4690


We need to change the array shape of each row of X to have it be parsable by TensorFlow.

In [12]:
def make_arrays(df_slice):
    convert = np.array(df_slice)
    output=[]
    i=0
    for row in convert:
        array = np.dstack(row)
        output.append(array)
        i+=1
    return np.array(output)

In [13]:
X_arr=make_arrays(X)

In [14]:
X_arr.shape

(928, 3, 300, 28)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X_arr, y, random_state=42, test_size=0.2)

# Location Prediction Model 

We're going to analyze these 3x300 waveforms similar to how images are processed: run a convolutional filter to do some feature analysis over a 3x2 kernel, then max pool over the XYZ axis, then convolute over a two second range.

After extended experimentation, I was never able to outperform the mean. NN's were tried predicting latitude and longitude seperately and these regressed to the mean even more quickly. Attempts to predict magnitude also didn't achieve much.

Below, we have a deliberately overfit model that we'll build an app around. I'm doing this for demo purposes it produces more interesting visual result. (Scattered results in the primary earthquake area will look more like more interesting predictions than having every earthquake predicted to be in the same mean spot). This model was trained for 428 iterations with a batch size of 16 before early stopping kicked in. As it's gratuitiously overfit, the model training history  isn't of much use and is excluded.

In [16]:
epicentre_model=load_model('epicentre_prediction')

In [17]:
epicentre_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 3, 300, 32)        5408      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 1, 300, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 1, 300, 64)        4160      
_________________________________________________________________
flatten_4 (Flatten)          (None, 19200)             0         
_________________________________________________________________
dense_8 (Dense)              (None, 2048)              39323648  
_________________________________________________________________
dropout_4 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 2)                

In [18]:
train_preds=epicentre_model.predict(X_train)
test_preds=epicentre_model.predict(X_test)

In [19]:
r2_score(y_train, train_preds)

0.7798691081389937

In [20]:
r2_score(y_test, test_preds)

-0.35514588882745

In [21]:
m = folium.Map(location=[15.8033, -96.1337],
              zoom_start=7)
for i, actual in y_train.iterrows():
    lat=actual['latitude']
    long=actual['longitude']
    folium.Circle(radius=10_000, 
                  location=[lat, long],  color='crimson', fill='crimson').add_to(m)

for pred in train_preds:
    lat=pred[0]
    long=pred[1]
    folium.Circle(radius=10_000, 
                  location=[lat, long],  color='purple', fill='purple').add_to(m)
m

In [22]:
m2 = folium.Map(location=[15.8033, -96.1337],
              zoom_start=7)
for i, actual in y_test.iterrows():
    lat=actual['latitude']
    long=actual['longitude']
    folium.Circle(radius=10_000, 
                  location=[lat, long],  color='crimson', fill='crimson').add_to(m2)

for pred in test_preds:
    lat=pred[0]
    long=pred[1]
    folium.Circle(radius=10_000, 
                  location=[lat, long],  color='purple', fill='purple').add_to(m2)
m2

In [43]:
model.save("epicentre_prediction")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: epicentre_prediction\assets


Experiments in magnitude prediction were similarly unfruitful and are omitted here.

In [27]:
The following preds dataframe is being assembled to be exported to the app.

SyntaxError: invalid syntax (<ipython-input-27-f75c13bb03d7>, line 1)

In [26]:
final_df['pred_lat']=train_preds[:, 0]

ValueError: Length of values (742) does not match length of index (928)

The following dataframe is to record the model's predictions and will be loaded into the app.

In [31]:
preds_df=final_df.loc[y_test.index, :]

In [32]:
preds_df[["pred_lat", "pred_long"]]= test_preds

In [36]:
preds_df.drop(columns=device_cols, inplace=True)

In [37]:
preds_df

,magnitude,latitude,longitude,depth,Referencia de localizacion,utc_datetime,outlier,state,state_lab,major,pred_lat,pred_long
813,4.3,17.5122,-94.4787,166.2,"53 km al SUR de MINATITLAN, VER",2020-05-09 05:22:37,0,VER,16,0,16.585371,-94.576782
718,4.4,15.8758,-95.2278,61.2,"34 km al SUR de SALINA CRUZ, OAX",2019-08-31 03:31:12,0,OAX,10,0,15.948578,-94.899345
34,5.3,15.5845,-95.0720,14.2,"67 km al SUR de SALINA CRUZ, OAX",2019-10-02 10:13:08,0,OAX,10,0,15.422470,-95.395119
426,4.5,17.6913,-95.0020,138.7,"22 km al SUR de SAYULA DE ALEMAN, VER",2019-10-28 02:35:25,0,VER,16,0,16.735538,-96.099182
209,4.8,14.0585,-92.5035,17.5,"78 km al SUROESTE de CD HIDALGO, CHIS",2018-04-01 12:13:00,0,CHIS,3,0,16.034916,-97.557617
...,...,...,...,...,...,...,...,...,...,...,...,...
443,4.5,16.3332,-94.9263,56.6,"15 km al SURESTE de JUCHITAN DE ZARAGOZA, OAX",2018-03-25 11:17:39,0,OAX,10,0,14.018781,-91.411728
7,6.0,16.2477,-97.7750,10.2,"31 km al SURESTE de PINOTEPA NACIONAL, OAX",2018-02-19 06:56:58,0,OAX,10,1,15.467715,-93.921509
63,5.1,18.3467,-101.4410,71.4,"65 km al SUROESTE de HUETAMO, MICH",2019-03-03 15:25:54,0,MICH,9,0,16.451990,-98.287033
394,4.5,19.0892,-108.5320,15.4,"384 km al SUROESTE de PUERTO VALLARTA, JAL",2018-02-19 18:16:03,0,JAL,8,0,16.131550,-96.728256


In [38]:
preds_df.drop(columns=['outlier', 'state_lab'], inplace=True)

In [39]:
preds_df

,magnitude,latitude,longitude,depth,Referencia de localizacion,utc_datetime,state,major,pred_lat,pred_long
813,4.3,17.5122,-94.4787,166.2,"53 km al SUR de MINATITLAN, VER",2020-05-09 05:22:37,VER,0,16.585371,-94.576782
718,4.4,15.8758,-95.2278,61.2,"34 km al SUR de SALINA CRUZ, OAX",2019-08-31 03:31:12,OAX,0,15.948578,-94.899345
34,5.3,15.5845,-95.0720,14.2,"67 km al SUR de SALINA CRUZ, OAX",2019-10-02 10:13:08,OAX,0,15.422470,-95.395119
426,4.5,17.6913,-95.0020,138.7,"22 km al SUR de SAYULA DE ALEMAN, VER",2019-10-28 02:35:25,VER,0,16.735538,-96.099182
209,4.8,14.0585,-92.5035,17.5,"78 km al SUROESTE de CD HIDALGO, CHIS",2018-04-01 12:13:00,CHIS,0,16.034916,-97.557617
...,...,...,...,...,...,...,...,...,...,...
443,4.5,16.3332,-94.9263,56.6,"15 km al SURESTE de JUCHITAN DE ZARAGOZA, OAX",2018-03-25 11:17:39,OAX,0,14.018781,-91.411728
7,6.0,16.2477,-97.7750,10.2,"31 km al SURESTE de PINOTEPA NACIONAL, OAX",2018-02-19 06:56:58,OAX,1,15.467715,-93.921509
63,5.1,18.3467,-101.4410,71.4,"65 km al SUROESTE de HUETAMO, MICH",2019-03-03 15:25:54,MICH,0,16.451990,-98.287033
394,4.5,19.0892,-108.5320,15.4,"384 km al SUROESTE de PUERTO VALLARTA, JAL",2018-02-19 18:16:03,JAL,0,16.131550,-96.728256


In [40]:
pickle.dump(preds_df, open("..\..\data\pickled_files\\predictions.p", "wb"))